# MATH 5671 Group3 Project
## Real Estate Valuation

**Hukai Luo, Xuanbo Huang, Tae Park**

*The inputs are as follows <br>
X1=the transaction date (for example, 2013.250=2013 March, 2013.500=2013 June, etc.) <br>
X2=the house age (unit: year) <br>
X3=the distance to the nearest MRT station (unit: meter) <br>
X4=the number of convenience stores in the living circle on foot (integer) <br>
X5=the geographic coordinate, latitude. (unit: degree) <br>
X6=the geographic coordinate, longitude. (unit: degree) <br>
The output is as follow <br>
Y= house price of unit area (10000 New Taiwan Dollar/Ping, where Ping is a local unit, 1 Ping = 3.3 meter squared) <br>*


*The input data set contains data about details of various houses. Based on the information provided, the goal is to come up with a model to predict median value of a given house in the area.*

## Load data

First there is a training feather contains 6 features and training labels contains the price combine the two excel and lord the data of the feathers and price.

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pandas as pd
sc= SparkContext()
sqlContext = SQLContext(sc)
house_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("/Users/luohukai/Desktop/Bigdata/Group3Project/training.csv")
house_df.show(6)

+------------+---------+------------+------+--------+---------+-----------+
|  trans_date|house_age|distance_mrt|stores|latitude|longitude|house_price|
+------------+---------+------------+------+--------+---------+-----------+
|      2013.5|     26.4|    335.5273|     6| 24.9796| 121.5414|       38.1|
|2013.4166667|      6.4|    90.45606|     9|24.97433| 121.5431|       59.5|
|2013.3333333|      3.9|    49.66105|     8|24.95836|121.53756|       56.8|
|      2013.5|     23.0|    3947.945|     0|24.94783|121.50243|       25.3|
|2013.3333333|     39.7|    333.3679|     9|24.98016|121.53932|       32.4|
|2013.1666667|     13.2|    750.0704|     2|24.97371|121.54951|       37.8|
+------------+---------+------------+------+--------+---------+-----------+
only showing top 6 rows



## Explore data

In [2]:
house_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
trans_date,289,2013.149653980968,0.2823817648239694,2012.6666667,2013.5833333
house_age,289,17.356747404844292,11.335506737460035,0.0,43.8
distance_mrt,289,1106.0805898961937,1270.742433103357,23.38284,6488.021
stores,289,4.041522491349481,2.919939669577361,0,10
latitude,289,24.969056678200687,0.012833229438307133,24.93207,25.01459
longitude,289,121.5331102422146,0.015300875869433324,121.47353,121.56627
house_price,289,38.33321799307961,13.824176325079815,11.2,117.5


Plot pairwise relationships in a dataset. As a rough guide, it provides us with an intuitive impression of the data.

In [3]:
import seaborn as sns
sns.set(style="ticks")
df = house_df.toPandas()
sns.pairplot(df)

The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that there is a strong positive correlation. When the coefficient is close to –1, it means that there is a strong negative correlation.

In [4]:
df.corr()

,trans_date,house_age,distance_mrt,stores,latitude,longitude,house_price
trans_date,1.000000,-0.026789,0.081076,-0.008615,0.038489,-0.061719,0.085179
house_age,-0.026789,1.000000,0.042041,-0.006156,0.035861,-0.076073,-0.239551
distance_mrt,0.081076,0.042041,1.000000,-0.620676,-0.584169,-0.800303,-0.674142
stores,-0.008615,-0.006156,-0.620676,1.000000,0.423986,0.449114,0.564233
latitude,0.038489,0.035861,-0.584169,0.423986,1.000000,0.423784,0.543654
longitude,-0.061719,-0.076073,-0.800303,0.449114,0.423784,1.000000,0.526381
house_price,0.085179,-0.239551,-0.674142,0.564233,0.543654,0.526381,1.000000


Prepare data for Machine Learning.

In [5]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['trans_date', 'house_age', 'distance_mrt', 'stores','latitude','longitude'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'house_price'])
vhouse_df.show(6)
train_df = vhouse_df
test_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("/Users/luohukai/Desktop/Bigdata/Group3Project/test_features.csv")
test_df = vectorAssembler.transform(test_df).select(['features'])

+--------------------+-----------+
|            features|house_price|
+--------------------+-----------+
|[2013.5,26.4,335....|       38.1|
|[2013.4166667,6.4...|       59.5|
|[2013.3333333,3.9...|       56.8|
|[2013.5,23.0,3947...|       25.3|
|[2013.3333333,39....|       32.4|
|[2013.1666667,13....|       37.8|
+--------------------+-----------+
only showing top 6 rows



## Linear Regression

In [6]:
from pyspark.ml.regression import LinearRegression

# Build the model
lr = LinearRegression(featuresCol = 'features', labelCol='house_price', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [4.369900121382684,-0.2551146024247613,-0.004445657936001385,0.9718688380688086,218.67565148018338,0.0]
Intercept: -14213.637049352814


Then use the model to predict the test data and compare with the real price.to evaluating the model.

In [7]:
# Evaluating the model on training data
lr_predictions = lr_model.transform(train_df)
lr_predictions.select("prediction","house_price","features").show(6)
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("R2: %f" % trainingSummary.r2)

+------------------+-----------+--------------------+
|        prediction|house_price|            features|
+------------------+-----------+--------------------+
|45.191696686018986|       38.1|[2013.5,26.4,335....|
| 52.78251927062411|       59.5|[2013.4166667,6.4...|
|  48.7733888096609|       56.8|[2013.3333333,3.9...|
|17.220974442165243|       25.3|[2013.5,23.0,3947...|
| 44.11802047399033|       32.4|[2013.3333333,39....|
| 40.08418444806921|       37.8|[2013.1666667,13....|
+------------------+-----------+--------------------+
only showing top 6 rows

RMSE: 8.949085
R2: 0.579482


R2 at 0.579 indicates that in our model, approximate 57.9% of the variability in “house price” can be explained using the model. It's not very good, we will try some other models in the bottom half of the project.

Root Mean Square Error (RMSE) is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are. We can compare the RMSE with the mean, min, max value of house_price.

In [8]:
train_df.describe().show()

+-------+------------------+
|summary|       house_price|
+-------+------------------+
|  count|               289|
|   mean| 38.33321799307961|
| stddev|13.824176325079815|
|    min|              11.2|
|    max|             117.5|
+-------+------------------+



After we created the model, let's use it to predict the test_features data and write dataframe to csv files named "Linear Regression Prediction.csv".

In [9]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","features").show(12)
lr_predictions = lr_predictions.select("prediction")
df = lr_predictions.toPandas()
df = df.round(1)
df.index += 1
df.to_csv("/Users/luohukai/Desktop/Bigdata/Group3Project/Linear Regression Prediction.csv",index=True,index_label="num")

+------------------+--------------------+
|        prediction|            features|
+------------------+--------------------+
| 40.89063915062252|[2012.8333333,10....|
|13.568364610426215|[2013.3333333,24....|
| 42.12474419017053|[2013.3333333,34....|
|13.381607476820136|[2013.3333333,25....|
|42.837319958853186|[2013.5,14.4,169....|
| 39.96450485170317|[2013.0833333,36....|
|40.691875930433525|[2013.25,35.8,170...|
| 34.72268917836118|[2013.0833333,34....|
|49.161584409745046|[2013.4166667,10....|
| 45.90773468961015|[2013.4166667,17....|
| 45.82964337429439|[2013.25,13.3,250...|
|  40.9628230769722|[2013.0833333,13....|
+------------------+--------------------+
only showing top 12 rows



## Decision tree regression

In [10]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'house_price')
dt_model = dt.fit(train_df)

# Evaluate model on test instances and compute test error
dt_predictions = dt_model.transform(train_df)
dt_predictions.select("prediction","house_price","features").show(6)
dt_evaluator = RegressionEvaluator(
    labelCol="house_price", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("RMSE: %g" % rmse)
dt_evaluator = RegressionEvaluator(
    labelCol="house_price", predictionCol="prediction", metricName="r2")
r2 = dt_evaluator.evaluate(dt_predictions)
print("R2: %g" % r2)

+------------------+-----------+--------------------+
|        prediction|house_price|            features|
+------------------+-----------+--------------------+
|41.505660377358495|       38.1|[2013.5,26.4,335....|
|55.900000000000006|       59.5|[2013.4166667,6.4...|
|             53.35|       56.8|[2013.3333333,3.9...|
|21.272727272727277|       25.3|[2013.5,23.0,3947...|
|41.505660377358495|       32.4|[2013.3333333,39....|
|41.505660377358495|       37.8|[2013.1666667,13....|
+------------------+-----------+--------------------+
only showing top 6 rows

RMSE: 6.1701
R2: 0.800101


R2 at 0.800 indicates that in our model, approximate 80.0% of the variability in “house price” can be explained using the model. Which indicates it's gonna be a better model compared to Linear model.

And the RMSE is also smaller than the linear model. Let's use it to predict the test_features data and write dataframe to csv files named "Decision tree regression Prediction.csv".

In [11]:
dt_predictions = dt_model.transform(test_df)
dt_predictions.select("prediction","features").show(12)
dt_predictions = dt_predictions.select("prediction")
df = dt_predictions.toPandas()
df = df.round(1)
df.index += 1
df.to_csv("/Users/luohukai/Desktop/Bigdata/Group3Project/Decision tree regression Prediction.csv",index=True,index_label="num")

+------------------+--------------------+
|        prediction|            features|
+------------------+--------------------+
| 66.02727272727272|[2012.8333333,10....|
|21.272727272727277|[2013.3333333,24....|
|47.289473684210535|[2013.3333333,34....|
|21.272727272727277|[2013.3333333,25....|
|33.166666666666664|[2013.5,14.4,169....|
|41.505660377358495|[2013.0833333,36....|
|47.289473684210535|[2013.25,35.8,170...|
|41.505660377358495|[2013.0833333,34....|
|55.900000000000006|[2013.4166667,10....|
|41.505660377358495|[2013.4166667,17....|
|47.289473684210535|[2013.25,13.3,250...|
|32.971428571428575|[2013.0833333,13....|
+------------------+--------------------+
only showing top 12 rows



## Random forest regression

In [12]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline

#Train a RF model.
rf = RandomForestRegressor(featuresCol="features" ,labelCol = 'house_price')
rf_model = rf.fit(train_df)

# Evaluate model on test instances and compute test error
rf_predictions = rf_model.transform(train_df)
rf_predictions.select('prediction', 'house_price', 'features').show(6)
rf_evaluator = RegressionEvaluator(
    labelCol="house_price", predictionCol="prediction", metricName="rmse")
rmse = rf_evaluator.evaluate(rf_predictions)
print("RMSE: %g" % rmse)
rf_evaluator = RegressionEvaluator(
    labelCol="house_price", predictionCol="prediction", metricName="r2")
r2 = rf_evaluator.evaluate(rf_predictions)
print("R2: %g" % r2)

+------------------+-----------+--------------------+
|        prediction|house_price|            features|
+------------------+-----------+--------------------+
| 43.48717717843245|       38.1|[2013.5,26.4,335....|
| 55.79716695174881|       59.5|[2013.4166667,6.4...|
| 51.15559966095081|       56.8|[2013.3333333,3.9...|
| 21.37603928667345|       25.3|[2013.5,23.0,3947...|
| 41.77447330947575|       32.4|[2013.3333333,39....|
|39.649901425772036|       37.8|[2013.1666667,13....|
+------------------+-----------+--------------------+
only showing top 6 rows

RMSE: 5.53185
R2: 0.839318


R2 at 0.834 indicates that in our model, approximate 83.4% of the variability in “house price” can be explained using the model. Which indicates it's better than decision tree model.

And the RMSE is also smaller than the decision tree model. Let's use it to predict the test_features data and write dataframe to csv files named "Random forest regression Prediction.csv".

In [13]:
rf_predictions = rf_model.transform(test_df)
rf_predictions.select("prediction","features").show(12)
rf_predictions = rf_predictions.select("prediction")
df = rf_predictions.toPandas()
df = df.round(1)
df.index += 1
df.to_csv("/Users/luohukai/Desktop/Bigdata/Group3Project/Random forest regression Prediction.csv",index=True,index_label="num")

+------------------+--------------------+
|        prediction|            features|
+------------------+--------------------+
|60.460439219380575|[2012.8333333,10....|
|18.436181394118435|[2013.3333333,24....|
| 47.25843185751544|[2013.3333333,34....|
|18.436181394118435|[2013.3333333,25....|
| 38.30144761315738|[2013.5,14.4,169....|
| 40.78089738526327|[2013.0833333,36....|
| 44.40396786965118|[2013.25,35.8,170...|
| 37.58001480557078|[2013.0833333,34....|
| 54.42066951019791|[2013.4166667,10....|
| 44.02957301176579|[2013.4166667,17....|
| 44.99080668544066|[2013.25,13.3,250...|
| 40.63270932993363|[2013.0833333,13....|
+------------------+--------------------+
only showing top 12 rows



## Gradient-boosted tree regression

Gradient-boosted tree regression performed the best on our data.

In [14]:
from pyspark.ml.regression import GBTRegressor

#Train a GBT model.
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'house_price', maxIter=10)
gbt_model = gbt.fit(train_df)

# Evaluate model on test instances and compute test error
gbt_predictions = gbt_model.transform(train_df)
gbt_predictions.select('prediction', 'house_price', 'features').show(6)
gbt_evaluator = RegressionEvaluator(
    labelCol="house_price", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("RMSE: %g" % rmse)
gbt_evaluator = RegressionEvaluator(
    labelCol="house_price", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator.evaluate(gbt_predictions)
print("R2: %g" % r2)

+------------------+-----------+--------------------+
|        prediction|house_price|            features|
+------------------+-----------+--------------------+
| 40.87056251705553|       38.1|[2013.5,26.4,335....|
|58.034505661918494|       59.5|[2013.4166667,6.4...|
| 55.42820938021139|       56.8|[2013.3333333,3.9...|
| 22.45522820182728|       25.3|[2013.5,23.0,3947...|
| 37.49716133229313|       32.4|[2013.3333333,39....|
| 39.32259574025981|       37.8|[2013.1666667,13....|
+------------------+-----------+--------------------+
only showing top 6 rows

RMSE: 3.83549
R2: 0.922756


R2 at 0.923 indicates that in our model, approximate 92.3% of the variability in “house price” can be explained using the model. Which indicates it's better than random forest regression.

And the RMSE is also smaller than the random forest regression. Let's use it to predict the test_features data and write dataframe to csv files named "Gradient-boosted tree regression Prediction.csv".

In [15]:
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select("prediction","features").show(12)
gbt_predictions = gbt_predictions.select("prediction")
df = gbt_predictions.toPandas()
df = df.round(1)
df.index += 1
df.to_csv("/Users/luohukai/Desktop/Bigdata/Group3Project/Gradient-boosted tree regression Prediction.csv",index=True,index_label="num")

+------------------+--------------------+
|        prediction|            features|
+------------------+--------------------+
| 76.45807587792638|[2012.8333333,10....|
| 21.21196441945712|[2013.3333333,24....|
| 43.48869956637004|[2013.3333333,34....|
| 21.21196441945712|[2013.3333333,25....|
|   35.530447441919|[2013.5,14.4,169....|
| 38.48070614489383|[2013.0833333,36....|
| 41.95445382782645|[2013.25,35.8,170...|
| 39.43941818337715|[2013.0833333,34....|
| 51.95425816831245|[2013.4166667,10....|
| 42.50226225247384|[2013.4166667,17....|
| 46.55911943657839|[2013.25,13.3,250...|
|33.126064022462984|[2013.0833333,13....|
+------------------+--------------------+
only showing top 12 rows

